### 臺中市空氣品質監測站即時監測資料 - 利用爬蟲技術擷取資料
**資料來源：[臺中市空氣品質監測站即時監測資料 — 台中市政府資料開放平台](https://opendata.taichung.gov.tw/dataset/6c6452ef-1e29-4e9e-b20e-e724c7e2959f)**

### <font color=#FF6600>CSV檔案爬蟲</font>


In [2]:
import csv
import os

from io import BytesIO
import requests # downloading with requests, and import the requests library
import urllib   # downloading with urllib, and import the urllib library
from urllib.request import urlretrieve  # used to copy a network object to a local file

# used to solve "urlopen error [SSL: CERTIFICATE_VERIFY_FAILED]" error
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#############################################################################################

############     定義align對齊函數    ############
def align(string,length):
    block = length - len(string)  # 計算限定長度需補齊多少的空格
    if block == 0:       #不需要空格補齊
        return string
    elif block < 0:      #限定的對齊長度小於字串傳入長度
        return None
    
    count = -1            #預留一格半形
    new_string = ''
    half_space = ' '     #'半形'空格
    space = '　'        #'全形'空格
    for i in string:
        codes = ord(i)      #將字符轉為ASCLL Code
        if codes <= 126:      #ASCLL小於126，代表佔半形空格(數字,符號,英文)
            count+=1           #紀錄最後須加上多少半形空格以將資料對齊
        new_string = new_string + i        #中文字(全形)不轉換
            
    return new_string + space*(block) + half_space*(count)


############     擷取csv並隨時儲存    ############
url = 'https://datacenter.taichung.gov.tw/swagger/OpenData/ab6ab38e-de1f-4718-915d-e81ea0761273'
urlretrieve(url,"dataset/csvdata")


###########   開檔時使用 utf-8-sig 的編碼方式來處理 BOM 字組頭的問題    ###########
count = 0
col_frame=[]

with open('dataset/csvdata', 'r', encoding = 'utf-8-sig', newline = '') as csvfile:
  
    plots = csv.reader(csvfile, delimiter = ',')   #利用'reader'並依照逗號格式區分
        
    for row in plots:
        if count == 0:
            for i in row:
                col_frame.append(i)              #將欄位標題分別append進入list
                count += 1                     #紀錄共有幾個欄位
                          
            print(','.join(col_frame).join('『』'))       #印出欄位標題，join並用『』括起來，以逗號間隔
            print('\n')
  
        print(align(row[0],10) + align(row[1],13) + align(row[2],12) + align(row[3],8) + align(row[4],12))
            
    csvfile.close()

『測站名稱,測值時間,監測項目,測值,狀態』


測站名稱　　　　　　測值時間　　　　　　　　　監測項目　　　　　　　　測值　　　　　　狀態　　　　　　　　　　
文山站　　　　　　　202304151600　           大氣壓力　　　　　　　　993.36　　     正常　　　　　　　　　　
文山站　　　　　　　202304151600　           甲烷　　　　　　　　　　2.19　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304151600　           ㄧ氧化碳　　　　　　　　0.47　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304151600　           非甲烷碳氫化合物　　　　0.1　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202304151600　           ㄧ氧化氮　　　　　　　　1.3　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202304151600　           二氧化氮　　　　　　　　9.1　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202304151600　           總氧氮化物　　　　　　　10.4　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304151600　           臭氧　　　　　　　　　　61.7　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304151600　           10u懸浮微粒　　　　　  27　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304151600　           2.5u懸浮微粒　　　　   20　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304151600　           雨量　　　　　　　　　　0　　　　　　　正常　　　　　　　　　　
文山站　　　　　　　202304151600　           相對溼度　　　　　　　　67.9　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304151600　           二氧化硫　　　　　　　　0.9　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202

### <font color=#0000FF>JSON檔案爬蟲</font>

                                                      

In [57]:
import json

from io import BytesIO
import requests # downloading with requests, and import the requests library
import urllib   # downloading with urllib, and import the urllib library
from urllib.request import urlretrieve  # used to copy a network object to a local file

#############################################################################################

############     定義align對齊函數    ############
def align(string,length):
    block = length - len(string)  # 計算限定長度需補齊多少的空格
    if block == 0:       #不需要空格補齊
        return string
    elif block < 0:      #限定的對齊長度小於字串傳入長度
        return None
    
    count = -1
    new_string = ''
    half_space = ' '     #'半形'空格
    space = '　'        #'全形'空格
    for i in string:
        codes = ord(i)      #將字符轉為ASCLL Code
        if codes <= 126:      #ASCLL小於126，代表佔半形空格(數字,符號,英文)
            count+=1           #紀錄最後須加上多少半形空格以將資料對齊
        new_string = new_string + i        #中文字(全形)不轉換
            
    return new_string + space*(block) + half_space*(count)


############     擷取json並隨時儲存    ############ 
url = 'https://datacenter.taichung.gov.tw/swagger/OpenData/ad8998ea-a749-40cb-80e5-6e001f262b7b'
urlretrieve(url,"dataset/jsondata")    

############     開啟檔案並依照排版輸出     ############ 
with open('dataset/jsondata', 'r', encoding = 'utf-8-sig', newline = '') as jsonfile:   #此處開啟json檔案

    data = json.load(jsonfile)   #運用關鍵字'load'將json轉為py檔案
    print(align('測站名稱',10) + align('測值時間',13) + align('監測項目',12) + align('測值',8) + align('狀態',12))
    for item in data:      # python 中的字典型式 儲存在Json格式中是以item呈現 
     
        print(align(item['測站名稱'],10) + align(item['測值時間'],13) + align(item['監測項目'],12) + align(item['測值'],8) + align(item['狀態'],12))
        
    jsonfile.close()

測站名稱　　　　　　測值時間　　　　　　　　　監測項目　　　　　　　　測值　　　　　　狀態　　　　　　　　　　
文山站　　　　　　　202304011300　           大氣壓力　　　　　　　　994.79　　     正常　　　　　　　　　　
文山站　　　　　　　202304011300　           甲烷　　　　　　　　　　0　　　　　　　維護　　　　　　　　　　
文山站　　　　　　　202304011300　           ㄧ氧化碳　　　　　　　　0.41　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304011300　           非甲烷碳氫化合物　　　　0　　　　　　　維護　　　　　　　　　　
文山站　　　　　　　202304011300　           ㄧ氧化氮　　　　　　　　1.4　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202304011300　           二氧化氮　　　　　　　　12　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304011300　           總氧氮化物　　　　　　　13.5　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304011300　           臭氧　　　　　　　　　　45.7　　　　   正常　　　　　　　　　　
文山站　　　　　　　202304011300　           10u懸浮微粒　　　　　  24　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304011300　           2.5u懸浮微粒　　　　   15　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304011300　           雨量　　　　　　　　　　0　　　　　　　正常　　　　　　　　　　
文山站　　　　　　　202304011300　           相對溼度　　　　　　　　89　　　　　　 正常　　　　　　　　　　
文山站　　　　　　　202304011300　           二氧化硫　　　　　　　　0.9　　　　　  正常　　　　　　　　　　
文山站　　　　　　　202304011300　           日照　　　　　　　　　　

### <font color=#0000FF>XML檔案爬蟲</font>
                                             

In [58]:
import xml.etree.ElementTree as et
#import xmltodict

from io import BytesIO
import requests # downloading with requests, and import the requests library
import urllib   # downloading with urllib, and import the urllib library
from urllib.request import urlretrieve  # used to copy a network object to a local file
#############################################################################################

############     定義align對齊函數    ############

def align(string,length):
    block = length - len(string)  # 計算限定長度需補齊多少的空格
    if block == 0:       #不需要空格補齊
        return string
    elif block < 0:      #限定的對齊長度小於字串傳入長度
        return None
    
    count = -1
    new_string = ''
    half_space = ' '     #'半形'空格
    space = '　'        #'全形'空格
    for i in string:
        codes = ord(i)      #將字符轉為ASCLL Code
        if codes <= 126:      #ASCLL小於126，代表佔半形空格(數字,符號,英文)
            count+=1           #紀錄最後須加上多少半形空格以將資料對齊
        new_string = new_string + i        #中文字(全形)不轉換
            
    return new_string + space*(block) + half_space*(count)


############     擷取xml並隨時儲存    ############
url = 'https://datacenter.taichung.gov.tw/swagger/OpenData/072e5e1d-fb39-4bda-847f-f42b090e2921'
urlretrieve(url,"dataset/xmldata")


############     分層結構樹(Element Tree)與利用子孫節點遍歷    ############
tree = et.ElementTree(file = 'dataset/xmldata')    #將xml file擷取成樹
root = tree.getroot()   #得到root element


############     以迴圈遍歷各 child 節點的子節點（孫節點）    ############  
print(align('測站名稱',14) + align('測值時間',13) + align('監測項目',13) + align('測值',12) + align('狀態',12))
for child in root:
         for grandchild in child:
                print(align(grandchild.text,13),end="")   #印出grandchild data的text類型
         print()

測站名稱　　　　　　　　　　測值時間　　　　　　　　　監測項目　　　　　　　　　測值　　　　　　　　　　狀態　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           大氣壓力　　　　　　　　　994.79　　　　　　　     正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           甲烷　　　　　　　　　　　0　　　　　　　　　　　　維護　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           ㄧ氧化碳　　　　　　　　　0.41　　　　　　　　　   正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           非甲烷碳氫化合物　　　　　0　　　　　　　　　　　　維護　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           ㄧ氧化氮　　　　　　　　　1.4　　　　　　　　　　  正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           二氧化氮　　　　　　　　　12　　　　　　　　　　　 正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           總氧氮化物　　　　　　　　13.5　　　　　　　　　   正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           臭氧　　　　　　　　　　　45.7　　　　　　　　　   正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           10u懸浮微粒　　　　　　  24　　　　　　　　　　　 正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           2.5u懸浮微粒　　　　　   15　　　　　　　　　　　 正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           雨量　　　　　　　　　　　0　　　　　　　　　　　　正常　　　　　　　　　　　
文山站　　　　　　　　　　202304011300　           相對溼度　　　　　　　　　89　　　　　　　　　　　 